# Take snapshots of time-series for each file and save them to disk

In [1]:
import os
import matplotlib.pyplot as plt

from db import connect_and_make_session, d_models
from plot import set_font_size
from shortcuts import get_processed_data_loaders


SAVE_DIR = 'data_snapshots'
LABELS = ('behav', 'gcamp')
BEHAV_VARIABLES = [
    'speed', 'ball_speed', 'v_ang', 'air_tube'
]
GCAMP_VARIABLES = [
    'G2S', 'G3S', 'G4S', 'G5S', 'G2D', 'G3D', 'G4D', 'G5D'
]

### Define function that saves data time-series to PNG

In [2]:
def trial_to_png(
    save_file, trial, variables, width=30, ax_height=3,
    lw=2, colors=None, y_lims=None):
    """
    Save the complete time-series data for several quantities for
    a trial to a single PNG image for efficient viewing.
    """
    if colors is None:
        colors = {}
    if y_lims is None:
        y_lims = {}
        
    for v in variables:
        if v not in colors:
            colors[v] = 'k'
        
    # make sure save directory exists
    save_dir = os.path.dirname(save_file)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    try:
        dl = trial.dl
    except:
        dl = get_processed_data_loaders([trial], vel_filt=None)[0]
    # get quantities from trial
    ts = dl.timestamp_gcamp
    
    data = {}
    for v in variables:
        try:
            data[v] = getattr(dl, v)
        except:
            raise KeyError('Variable "{}" not found for trial.'.format(v))
    
    # arrange figure
    fig_size = (width, ax_height*len(variables))
    fig, axs = plt.subplots(
        len(variables), 1, figsize=fig_size, tight_layout=True, squeeze=False)
    
    for ax, v in zip(axs[:, 0], variables):

        color = colors[v]
        ax.plot(ts, data[v], color=color, lw=lw)
        ax.grid()
        
        ax.set_xlabel('t (s)')
        ax.set_ylabel(v)
        
        ax.set_xlim(ts[0], ts[-1])
        if v in y_lims:
            ax.set_ylim(y_lims[v])
    
        set_font_size(ax, 16)
        
    # save png
    fig.savefig(save_file)
    plt.close(fig)
    
    return save_file

### Make and save images for each trial

In [3]:
session = connect_and_make_session()

trials = session.query(d_models.Trial).all()

for trial in trials:
    print('Saving data snapshot for trial "{}"...'.format(trial.name))
    save_dir = os.path.join(
        SAVE_DIR, trial.fly,
        '{} ({})'.format(trial.name, trial.expt))
    
    for label, vs in zip(LABELS, [BEHAV_VARIABLES, GCAMP_VARIABLES]):
        save_path = os.path.join(save_dir, '{}.png'.format(label))
        
        trial_to_png(save_path, trial, vs)
        
        print('Snapshot saved at "{}".'.format(save_path))

Saving data snapshot for trial "20160904.Fly4.2"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.3"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.6"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.6 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly4.7"...
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160904.Fly4/20160904.Fly4.7 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160904.Fly5.3"...
Snapshot saved at "d

Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly6.4"...
Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.4 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly6/20160905.Fly6.4 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.1"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.1 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.1 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.2"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160905.Fly7.3"...
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160905.Fly7/20160905.Fly7.3 (closed_

Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.3"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.3 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.4"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.4 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.4 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.5"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.5 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.5 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20160906.Fly9.6"...
Snapshot saved at "data_snapshots/20160906.Fly9/20160906.Fly9.6 (closed_

Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.4 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly1.8"...
Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.8 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly1/20170313.Fly1.8 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.1"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.1 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.2"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.2 (closed_loop)/gcamp.png".
Saving data snapshot for trial "20170313.Fly3.3"...
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_loop)/behav.png".
Snapshot saved at "data_snapshots/20170313.Fly3/20170313.Fly3.3 (closed_

Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.3 (driven_sinusoidal)/gcamp.png".
Saving data snapshot for trial "20170117.Fly1.4"...
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.4 (driven_sinusoidal)/behav.png".
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.4 (driven_sinusoidal)/gcamp.png".
Saving data snapshot for trial "20170117.Fly1.5"...
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.5 (driven_sinusoidal)/behav.png".
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.5 (driven_sinusoidal)/gcamp.png".
Saving data snapshot for trial "20170117.Fly1.6"...
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.6 (driven_sinusoidal)/behav.png".
Snapshot saved at "data_snapshots/20170117.Fly1/20170117.Fly1.6 (driven_sinusoidal)/gcamp.png".
Saving data snapshot for trial "20170118.Fly2.4"...
Snapshot saved at "data_snapshots/20170118.Fly2/20170118.Fly2.4 (driven_sinusoidal)/behav.png".
Snapshot saved at "data_